In [134]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
investment_data=pd.read_excel("investments.xlsx")

In [135]:
X = investment_data.drop(columns=['Investment'])
y = investment_data['Investment']

In [136]:
# Encode categorical variables
label_encoder = LabelEncoder()
X['Type'] = label_encoder.fit_transform(X['Type'])
X['Potential Return'] = label_encoder.fit_transform(X['Potential Return'])

In [137]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [138]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_scaled, y)

RandomForestClassifier(random_state=42)

In [139]:
with open('investment_model.pkl', 'wb') as file:
    pickle.dump(model, file)